In [1]:
import sys
#sys.path.insert(1, r"folder where you put files from zenodo")

# from bayes_multi_order_model_1 import BayesMultiOrderNetworkModel
from bayes_multi_order_model_1 import opt_order_bayes_factors
from bayes_multi_order_model_1 import ML_loglh_and_DoF_of_BMONs
from bayes_multi_order_model_1 import AIC, BIC, p_value

from paths import Paths
from constraint import NetworkConstraint

import numpy as np



In [2]:
paths = Paths()

file1 = r"MSNBC.ngram"
paths.update_from_ngram(file1, splitter = ",", last_entry_frequency = False)
paths.print_stats()

total number of paths: 31790
length | total #,  # uniques | min #, max # |
     8 | 7817.00,    6951.00 |  1.00, 75.00 |
     9 | 5212.00,    4965.00 |  1.00, 32.00 |
    10 | 4204.00,    4018.00 |  1.00, 20.00 |
    11 | 2819.00,    2745.00 |  1.00, 18.00 |
    12 | 2418.00,    2366.00 |  1.00, 11.00 |
    13 | 1689.00,    1651.00 |  1.00,  9.00 |
    14 | 1390.00,    1357.00 |  1.00,  7.00 |
    15 |  951.00,     937.00 |  1.00,  9.00 |
    16 |  890.00,     881.00 |  1.00,  4.00 |
    17 |  676.00,     671.00 |  1.00,  2.00 |
    18 |  598.00,     591.00 |  1.00,  4.00 |
    19 |  433.00,     430.00 |  1.00,  4.00 |
    20 |  375.00,     370.00 |  1.00,  3.00 |
    21 |  288.00,     288.00 |  1.00,  1.00 |
    22 |  260.00,     260.00 |  1.00,  1.00 |
    23 |  187.00,     187.00 |  1.00,  1.00 |
    24 |  166.00,     166.00 |  1.00,  1.00 |
    25 |  146.00,     146.00 |  1.00,  1.00 |
    26 |  130.00,     129.00 |  1.00,  2.00 |
    27 |  110.00,     109.00 |  1.00,  2.00 |
    2

# The graph constraint:
We assume that all edges were traversed. The assumption is not good, but it is the best we can do with this data.

In [3]:
edges = set()
nodes = set()
for length in paths.lengths():
    for path in paths.paths_of_length(length):
        for en,node in enumerate(path[1:]):
            edge = (path[en],node)
            edges.add(edge)
            # edges.add((edge[1], edge[0]))
            nodes.update(set(edge))
            

In [4]:
len(edges)

270

In [5]:
len(nodes)

17

In [6]:
topo_const = NetworkConstraint(nodes,edges)

# The Order detection
## Optimal order using MLE methods

In [7]:
max_order = 3

In [8]:
ml_log_lh, dof, n_obs = ML_loglh_and_DoF_of_BMONs(
    paths = paths,
    max_order = max_order,
    nodes = nodes,
    constraint = topo_const
)

### AIC

In [9]:
for i in range(max_order+1):
    print(i, AIC(ml_log_lh[i], dof[i]))
print("MIN:", min(range(max_order +1), key = lambda i: AIC(ml_log_lh[i], dof[i])))

0 2059889.8530055059
1 1780381.034246967
2 1513925.9939165465
3 1548158.1365096106
MIN: 2


### BIC

In [10]:
for i in range(max_order+1):
    print(i, BIC(ml_log_lh[i], dof[i], n_obs[i]))
print("MIN:", min(range(max_order + 1), key = lambda i: BIC(ml_log_lh[i], dof[i], n_obs[i])))

0 2060065.1643701894
1 1783328.4565657105
2 1560920.3966120512
3 2294973.593101569
MIN: 2


### Likelihood ratio test

In [11]:
for i in range(1, max_order+1):
    print(f"reject {i-1} for {i}? p-value", p_value( ml_log_lh[i-1], ml_log_lh[i], dof[i-1], dof[i]))

reject 0 for 1? p-value 0.0
reject 1 for 2? p-value 0.0
reject 2 for 3? p-value 0.0


## Bayes factor method

In [12]:
_, bayes_log_lh = opt_order_bayes_factors(
    paths = paths, 
    max_order = max_order, 
    nodes = nodes,
    constraint = topo_const
)

In [13]:
for i in range(1, max_order+1):
    print()
    passed = True
    for j in range(i):
#         print(i, j)
#         print(2*(bayes_log_lh[i] - bayes_log_lh[j]))
        passed = passed and ((bayes_log_lh[i] - bayes_log_lh[j]) > np.log(150))
#         print(2*(bayes_log_lh[i] - bayes_log_lh[j]) > 10)
#         print()
    if passed:
        print(f"Order {i} is better than all lower.")
    else:
        print(f"Order {i} is NOT better than all lower.")
        


Order 1 is better than all lower.

Order 2 is better than all lower.

Order 3 is NOT better than all lower.


In [14]:
print(n_obs)

defaultdict(<class 'int'>, {0: 423776, 1: 423776, 2: 423776, 3: 423776})
